# Third exercise: plot the monthly number of searches for flights arriving at Málaga, Madrid or Barcelona

For the arriving airport, you can use the Destination column in the searches file. Plot a curve for Málaga, another one for Madrid, and another one for Barcelona, in the same figure. Bonus point: Solving this problem using pandas (instead of any other approach) 

In [87]:
val searches = sc.textFile("searches.csv.bz2")
val cleandata = searches.map(line => line.split("\\^").map(_.trim))
val firstRow = cleandata.first
val data = cleandata.filter( line => !line.contains(firstRow(0)) )
data.count

20390198

In [88]:
val numberFields = firstRow.length   // number of columns excpected for each line
val filteredData = data.filter( line => line.length >= numberFields) 
filteredData.first
 

Array(2013-01-01, 20:25:57, MPT, 624d8c3ac0b3a7ca03e3c167e0f48327, DE, TXL, AUH, 1, 2, TXL, AUH, 2013-01-26, D2, "", AUH, TXL, 2013-02-02, D2, "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", 1ASIWS, 0, 0, 0, d41d8cd98f00b204e9800998ecf8427e, FRA)

In [99]:
val destination = firstRow.indexOf("Destination") // index of arrival airport column in csv data file
val date = firstRow.indexOf("Date")           // index of number of passangers column in csv data file

val keyValRDD = filteredData.map{ x => (x(destination), x(date) ) }
keyValRDD.count

val madridSearches = keyValRDD.filter( kv => kv._1 == "MAD" )
madridSearches.first

(MAD,2013-01-01)

In [69]:
import org.apache.spark.rdd.RDD

def getSearchesPerMonth( searches: RDD[(String, java.util.Date)], month: Int) : Long =
{
    return searches.filter( kv=> kv._2.getMonth() == month ).count
}



In [111]:
def getSearchesForAirport( searches: RDD[(String, String)], airport: String) : RDD[(String, java.util.Date)] =
{
    val searchesAirport = searches.filter( kv => kv._1 == airport )
    val format = new java.text.SimpleDateFormat("yyyy-MM-dd")
    val returnValue = searchesAirport.mapValues( v => format.parse( v ));
    return returnValue
}

In [114]:

val madridSearchesDate = getSearchesForAirport(keyValRDD, "MAD")
madridSearchesDate.first
//val madridSearchesMonth = madridSearchesDate.filter( kv=> kv._2.getMonth() == 0 )
//madridSearchesMonth.first

Name: Syntax Error.
Message: 
StackTrace: 

In [117]:
val monthNumberToName = Array( "Jan","Feb", "Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec")
  
//val madridSearches = keyValRDD.filter( kv => kv._1 == "MAD" )
//val madridSearchesDate = madridSearches.mapValues( v => format.parse( v ))

val madridSearches = getSearchesForAirport(keyValRDD, "MAD")
val barcellonaSearches = getSearchesForAirport(keyValRDD, "BCN" )
val malagaSearches = getSearchesForAirport( keyValRDD, "AGP" )

println( "Month\tMalaga\tMadrid\tBarcellona")

for( month <- 0 to monthNumberToName.length-11)
{
    println( monthNumberToName(month) 
    + "\t" + getSearchesPerMonth(malagaSearches, month) 
    + "\t" + getSearchesPerMonth(madridSearches, month)
    + "\t" + getSearchesPerMonth(barcellonaSearches, month) 
    )
    
}
      

Month	Malaga	Madrid	Barcellona
Jan	9633	24258	29469
Feb	8379	22800	28329


In [20]:
val monthFormat = new java.text.SimpleDateFormat("MM-dd-yyyy");
val madridSearchesDateMonth = madridSearches.map(kv => monthFormat.format( kv._2 ))
madridSearchesDateMonth.first

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 0 in stage 37.0 failed 1 times, most recent failure: Lost task 0.0 in stage 37.0 (TID 129, localhost): java.lang.IllegalArgumentException: Cannot format given Object as a Date
	at java.text.DateFormat.format(DateFormat.java:310)
	at java.text.Format.format(Format.java:157)
	at $line157.$read$$iwC$$iwC$$iwC$$iwC$$anonfun$1.apply(<console>:43)
	at $line157.$read$$iwC$$iwC$$iwC$$iwC$$anonfun$1.apply(<console>:43)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:328)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:312)
	at scala.collection.Iterator$class.foreach(Iterator.scala:727)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1157)
	at scala.collection.generic.Growable$class.$plus$plus$eq(Growable.scala:48)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:103)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:47)
	at scala

In [14]:
object Utils {
    
    /**
    def getAirportRDD( data: RDD[Array[String]], airport: String) : array[long] = 
    {
    
        val numberSearches = Array(1, 2, 3)
        return numberSearches
    }
    */
    
    def getAirport( airport: String) : Array[Int] = 
    {
    
        val numberSearches = Array(1, 2, 3)
        return numberSearches
    }
}

val numbSearches = data.map( Utils.getAirport("MAD").toString )


Name: Compile Error
Message: <console>:31: error: type mismatch;
 found   : () => String
 required: Array[String] => ?
       val numbSearches = data.map( Utils.getAirport("MAD").toString )
                                                            ^
StackTrace: 

In [ ]:
//val madridSearches = data.map( line => line.contains("MAD") )
//numbSearches.count